## Linking the pseudopeople Census and ACS datasets

In this tutorial we will configure and link two realistic, simulated datasets generated by the [pseudopeople Python package](https://pseudopeople.readthedocs.io/en/latest/index.html). These datasets reflect a fictional sample population of ~10,000 simulants living in Anytown, WA, but pseudopeople can also generate [datasets](https://pseudopeople.readthedocs.io/en/latest/datasets/index.html) about two larger fictional populations, one simulating the state of Rhode Island, and the other simulating the entire United States.

Here we will link Anytown's 2020 [Census dataset](https://pseudopeople.readthedocs.io/en/latest/datasets/index.html#us-decennial-census) to its 2020 [ACS dataset](https://pseudopeople.readthedocs.io/en/latest/datasets/index.html#american-community-survey-acs). We expect every person surveyed in the smaller ACS dataset to also be counted in the census data, so we will aim to link every simulant in the ACS to the same simulant in the Census.

This tutorial is adapted from the [Febrl4 linking example](https://moj-analytical-services.github.io/splink/demos/examples/duckdb/febrl4.html).

<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/master/docs/demos/examples/duckdb/febrl4.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install splink
# !pip install pseuodopeople

### Configuring pseudopeople, exploring data and defining the model


Pseudopeople is designed to generate realistic data which is useful for testing entity resolution methods in that reflects the challenges of linking actual datasets. This requires adding [noise](https://pseudopeople.readthedocs.io/en/latest/noise/index.html) to the data in the form of various types of errors that real survey respondents may make. The default frequencies of each type of noise in the dataset can be configured for each column.

Because the ACS dataset is small and therefore has less opportunities for noise to create linkage challenges,  let's increase the noise for the `race_ethnicity` and `last_name` columns in both datasets from their default values. For `race_ethnicity` we increase the frequency with which respondents choose the response option to that survey question, and for `last_name` we increase both the frequency of respondents which may have last name typos, and the probability that each character will be typed incorrectly for those respondents. See [here](https://pseudopeople.readthedocs.io/en/latest/noise/column_noise.html#id16) for more details.


In [2]:
import pseudopeople as psp
import numpy as np

config_census = {
    'decennial_census': { # Dataset
        'column_noise': { # "Choose the wrong option" is in the column-based noise category
            'race_ethnicity': { # Column
                'choose_wrong_option': { # Noise type
                    'cell_probability': 0.3, # Parameter (and value)
                },
            },
            'last_name': { # Column
                'make_typos': { # Noise type
                    'cell_probability': 0.1, # Parameter (and value)
                    'token_probability': .15
                },
            },
        },
    },
}
config_acs = {
    'american_community_survey': { # Dataset
        'column_noise': { # "Choose the wrong option" is in the column-based noise category
            'race_ethnicity': { # Column
                'choose_wrong_option': { # Noise type
                    'cell_probability': 0.3, # Parameter (and value)
                },
            },
            'last_name': { # Column
                'make_typos': { # Noise type
                    'cell_probability': 0.1, # Parameter (and value)
                    'token_probability': .15
                },
            },
        },
    },
}

Next, lets get the data ready for splink. Note that the Census data has about 10,000 rows, while the ACS data has only 74.

In [3]:
import pandas as pd
census = psp.generate_decennial_census(config=config_census)
acs = psp.generate_american_community_survey(config=config_acs)

# drop columns that are not in both datasets
census = census.drop("year", axis=1)
acs = acs.drop("survey_date", axis=1)

# uniquely identify each row in the datasets
census["id"] = census.index
acs["id"] = len(census.index) + acs.index

# create unique ID to simulant_id lookup table
simulant_lookup = pd.concat([census[["id","simulant_id"]], acs[["id","simulant_id"]]]).reset_index(drop=True).set_index('id')

def prepare_data(data):
    # concatenate address fields, setting the new field to NaN
    # if any address fields besides unit number are missing
    data["unit_number"] = data["unit_number"].fillna("")

    columns_to_concat = ["street_number", "street_name", "unit_number", "city", "state", "zipcode"]

    data["has_addr_nan"] = data[columns_to_concat].isna().any(axis=1)

    data[columns_to_concat] = data[columns_to_concat].astype(str)
    data["address"] = data[columns_to_concat].agg(' '.join, axis=1)
    data.loc[data["has_addr_nan"], "address"] = np.nan
    data = data.drop("has_addr_nan", axis=1)

    return data

dfs = [prepare_data(dataset) for dataset in [census, acs]]

display(dfs[0])
display(dfs[1])



,simulant_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,id,address
0,0_2,0_7,Diana,P,Kofron,25,05/06/1994,5112,145th st,,Anytown,WA,00000,Household,Reference person,Female,Asian,0,5112 145th st Anytown WA 00000
1,0_3,0_7,Anna,A,Kofron,25,09/29/1994,5112,145th st,,Anytown,WA,00000,Household,Other relative,Female,White,1,5112 145th st Anytown WA 00000
2,0_923,0_8033,Gerald,R,Butler,76,11/03/1943,1130,mallory ln,,Anytown,WA,00000,Household,Reference person,Male,Black,2,1130 mallory ln Anytown WA 00000
3,0_2641,0_1066,Loretta,T,Carley,61,07/71/1958,nan,delacorte dr,,Anytown,WA,00000,Household,Reference person,Female,White,3,NaN
4,0_2801,0_1138,Richard,R,Jones,73,03/03/1947,950,caribou lane,,Anytown,WA,00000,Household,Reference person,Male,White,4,950 caribou lane Anytown WA 00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10226,0_11994,0_8051,Lauren,H,Consul,17,10/25/2002,3304,ethan allen way,unit 200,Anytown,WA,00000,Household,Other relative,Female,White,10226,3304 ethan allen way unit 200 Anytown WA 00000
10227,0_19693,0_6152,Johana,M,Huang,20,08/04/1999,1095,ernst st,,Anytown,WA,00000,Household,Other relative,Female,Asian,10227,1095 ernst st Anytown WA 00000
10228,0_19556,0_2064,Benjamin,F,Allen,19,02/26/Z001,2002,203rd pl se,,Anytown,WA,00000,Household,Other relative,Male,AIAN,10228,2002 203rd pl se Anytown WA 00000
10229,0_19579,0_1802,Brielle,L,Gonzalez,19,11/27/2000,233,saint peters road,,Anytown,WA,00000,Household,Other relative,Female,Latino,10229,233 saint peters road Anytown WA 00000


,simulant_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,id,address
0,0_12123,0_4898,Jamie,N,Lai,NaN,07/04/1977,820,cameron road,,Anytown,WA,nan,Other institutional,Opposite-sex spouse,Female,White,10231,NaN
1,0_12125,0_4898,Anastasia,S,Lai,47,07/04/1977,820,cameron road,,Anytown,WA,00000,Household,Biological child,Female,Asian,10232,820 cameron road Anytown WA 00000
2,0_20535,0_4898,Ronin,K,Lai,16,08/04/2020,820,nan,,Anytown,WA,00000,Household,Biological child,Male,Asian,10233,NaN
3,0_727,0_3,Kari,L,Henderson,36,05/05/1983,8203,west farwell avenue,,Anytown,WA,00000,College,Noninstitutionalized group quarters population,Female,White,10234,8203 west farwell avenue Anytown WA 00000
4,0_1928,0_3,Lucille,C,Spicer,81,12/10/1938,8203,west farwell avenue,,Anytown,WA,00000,College,Noninstitutionalized group quarters population,Female,Black,10235,8203 west farwell avenue Anytown WA 00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,0_19709,0_3,Erik,G,Rabe,23,09/21/1996,8203,west farwell avenue,,Anytown,WA,00000,College,Noninstitutionalized group quarters population,Male,Black,10300,8203 west farwell avenue Anytown WA 00000
70,0_20027,0_3,Bryce,J,Lam,21,08/17/1998,8203,west farwell avenue,,Anytown,WA,00000,College,Noninstitutionalized group quarters population,Male,Asian,10301,8203 west farwell avenue Anytown WA 00000
71,0_20329,0_3,Zachary,J,Collier,17,01/20/2003,8203,west farwell avenue,,Anytown,WA,00000,College,Noninstitutionalized group quarters population,Male,AIAN,10302,8203 west farwell avenue Anytown WA 00000
72,0_2895,0_1179,Justin,J,Sass,33,09/13/1987,111-24,gonzalez dr,apt # 5002,nan,WA,00000,Household,Reference person,Male,White,10303,NaN


Next, to better understand which variables will prove useful in linking, we have a look at how populated each column is, as well as the distribution of unique values within each


In [4]:
from splink import DuckDBAPI, Linker, SettingsCreator

basic_settings = SettingsCreator(
    unique_id_column_name="id",
    link_type="link_only",
)

linker = Linker(dfs, basic_settings, db_api=DuckDBAPI(), input_table_aliases=["census", "acs"])

It's usually a good idea to perform exploratory analysis on your data so you understand what's in each column and how often it's missing


In [5]:
from splink.exploratory import completeness_chart

completeness_chart(dfs, db_api=DuckDBAPI())

alt.LayerChart(...)

In [6]:
from splink.exploratory import profile_columns

profile_columns(dfs, db_api=DuckDBAPI())

alt.VConcatChart(...)

Next let's come up with some candidate blocking rules, which define which record comparisons are generated, and have a look at how many comparisons each will generate.

For blocking rules that we use in prediction, our aim is to have the union of all rules cover all true matches, whilst avoiding generating so many comparisons that it becomes computationally intractable - i.e. each true match should have at least _one_ of the following conditions holding.


In [7]:
from splink import DuckDBAPI, block_on
from splink.blocking_analysis import (
    cumulative_comparisons_to_be_scored_from_blocking_rules_chart,
)

blocking_rules = [
    block_on("first_name"),
    block_on("last_name"),
    block_on("date_of_birth"),
    block_on("street_name"),
    block_on("age", "sex"),
    block_on("age", "race_ethnicity"),
    block_on("age", "middle_initial"),
    block_on("street_number"),
    block_on("middle_initial", "sex", "race_ethnicity")
]


db_api = DuckDBAPI()
cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=dfs,
    blocking_rules=blocking_rules,
    db_api=db_api,
    link_type="link_only",
    unique_id_column_name="id",
    source_dataset_column_name="source_dataset",
)

alt.Chart(...)

For columns like `age` that would create too many comparisons, we combine them with one or more other columns which would also create too many comparisons if blocked on alone.


Now we get the full settings by including the blocking rules, as well as deciding the actual comparisons we will be including in our model.

For some columns for which certain comparison levels are not observed, I've gone ahead and removed them from the thresholds.

Since we know there should exist one simulant in the census for each simulant in the ACS, we can set `probability_two_random_records_match` using the sizes of each dataset.

In [8]:
import splink.comparison_library as cl


settings = SettingsCreator(
    unique_id_column_name="id",
    link_type="link_only",
    blocking_rules_to_generate_predictions=blocking_rules,
    comparisons=[
        cl.NameComparison("first_name", jaro_winkler_thresholds=[.92, .88]).configure(
            term_frequency_adjustments=True), # nothing obesrved for .7
        cl.ExactMatch("middle_initial").configure(term_frequency_adjustments=True),
        cl.NameComparison("last_name", jaro_winkler_thresholds=[.92]).configure(
            term_frequency_adjustments=True), # nothing observed for .88 or .7
        cl.DamerauLevenshteinAtThresholds("date_of_birth", distance_threshold_or_thresholds=[1]),
        cl.DamerauLevenshteinAtThresholds("address").configure(term_frequency_adjustments=True),
        cl.ExactMatch("sex"),
        cl.ExactMatch("race_ethnicity").configure(term_frequency_adjustments=True),
    ],
    retain_intermediate_calculation_columns=True,
    probability_two_random_records_match=dfs[1].size/dfs[0].size
)


linker = Linker(dfs, settings, db_api=DuckDBAPI())

### Estimating model parameters


We need to furnish our models with parameter estimates so that we can generate results. We will focus on the detailed model, generating the values for the simple model at the end


Next we estimate `u` and `m` values for each comparison, so that we can move to generating predictions


In [9]:
# We generally recommend setting max pairs higher (e.g. 1e7 or more)
# But this will run faster for the purpose of this demo
linker.training.estimate_u_using_random_sampling(max_pairs=1e6)

You are using the default value for `max_pairs`, which may be too small and thus lead to inaccurate estimates for your model's u-parameters. Consider increasing to 1e8 or 1e9, which will result in more accurate estimates, but with a longer run time.
----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - middle_initial (no m values are trained).
    - last_name (no m values are trained).
    - date_of_birth (no m values are trained).
    - address (no m values are trained).
    - sex (no m values are trained).
    - race_ethnicity (no m values are trained).


When training the `m` values using expectation maximisation, we need somre more blocking rules to reduce the total number of comparisons. For each rule, we want to ensure that we have neither proportionally too many matches, or too few.

We must run this multiple times using different rules so that we can obtain estimates for all comparisons - if we block on e.g. `date_of_birth`, then we cannot compute the `m` values for the `date_of_birth` comparison, as we have only looked at records where these match.


In [10]:
session_dob = (
    linker.training.estimate_parameters_using_expectation_maximisation(
        block_on("date_of_birth"), estimate_without_term_frequencies=True
    )
)
session_pc = (
    linker.training.estimate_parameters_using_expectation_maximisation(
        block_on("last_name"), estimate_without_term_frequencies=True
    )
)
session_pc = (
    linker.training.estimate_parameters_using_expectation_maximisation(
        block_on("first_name"), estimate_without_term_frequencies=True
    )
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."date_of_birth" = r."date_of_birth"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - middle_initial
    - last_name
    - address
    - sex
    - race_ethnicity

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - date_of_birth

Iteration 1: Largest change in params was -0.464 in the m_probability of race_ethnicity, level `Exact match on race_ethnicity`
Iteration 2: Largest change in params was -0.000364 in the m_probability of address, level `All other comparisons`
Iteration 3: Largest change in params was -6.66e-06 in the m_probability of first_name, level `All other comparisons`

EM converged after 3 iterations

Your model is not yet fully trained. Missing estimates for:
    - date_of_birth (no m values are trained).

----- Starting EM training session -----

Estimating the m

If we wish we can have a look at how our parameter estimates changes over these training sessions


In [11]:
session_dob.m_u_values_interactive_history_chart()

alt.HConcatChart(...)

For variables that aren't used in the `m`-training blocking rules, we have two estimates --- one from each of the training sessions (see for example `address`). We can have a look at how the values compare between them, to ensure that we don't have drastically different values, which may be indicative of an issue.


In [12]:
linker.visualisations.parameter_estimate_comparisons_chart()

alt.Chart(...)

In [13]:
import json
# we can have a look at the full settings if we wish, including the values of our estimated parameters:
print(json.dumps(linker._settings_obj.as_dict(), indent=2))
# we can also get a handy summary of of the model in an easily readable format if we wish:
print(linker._settings_obj.human_readable_description)
# (we suppress output here for brevity)

{
  "link_type": "link_only",
  "probability_two_random_records_match": 0.007232919558205454,
  "retain_matching_columns": true,
  "retain_intermediate_calculation_columns": true,
  "additional_columns_to_retain": [],
  "sql_dialect": "duckdb",
  "linker_uid": "jgzf5fso",
  "em_convergence": 0.0001,
  "max_iterations": 25,
  "bayes_factor_column_prefix": "bf_",
  "term_frequency_adjustment_column_prefix": "tf_",
  "comparison_vector_value_column_prefix": "gamma_",
  "unique_id_column_name": "id",
  "source_dataset_column_name": "source_dataset",
  "blocking_rules_to_generate_predictions": [
    {
      "blocking_rule": "l.\"first_name\" = r.\"first_name\"",
      "sql_dialect": "duckdb"
    },
    {
      "blocking_rule": "l.\"last_name\" = r.\"last_name\"",
      "sql_dialect": "duckdb"
    },
    {
      "blocking_rule": "l.\"date_of_birth\" = r.\"date_of_birth\"",
      "sql_dialect": "duckdb"
    },
    {
      "blocking_rule": "l.\"street_name\" = r.\"street_name\"",
      "sql_di

We can now visualise some of the details of our model. We can look at the match weights, which tell us the relative importance for/against a match for each of our comparsion levels.


In [14]:
linker.visualisations.match_weights_chart()

alt.VConcatChart(...)

As well as the match weights, which give us an idea of the overall effect of each comparison level, we can also look at the individual `u` and `m` parameter estimates, which tells us about the prevalence of coincidences and mistakes (for further details/explanation about this see [this article](https://www.robinlinacre.com/maths_of_fellegi_sunter/)). We might want to revise aspects of our model based on the information we ascertain here.

Note however that some of these values are very small, which is why the match weight chart is often more useful for getting a decent picture of things.


In [15]:
linker.visualisations.m_u_parameters_chart()

alt.HConcatChart(...)

It is also useful to have a look at unlinkable records - these are records which do not contain enough information to be linked at some match probability threshold. We can figure this out be seeing whether records are able to be matched with themselves.

We have low column missingness, so all of our records are linkable.


In [16]:
linker.evaluation.unlinkables_chart()

alt.LayerChart(...)

### Predictions

In [17]:
predictions = linker.inference.predict() # include all match_probabilities
df_predictions = predictions.as_pandas_dataframe()
display(df_predictions.head(2))

Blocking time: 0.09 seconds
Predict time: 0.83 seconds


,match_weight,match_probability,source_dataset_l,source_dataset_r,id_l,id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,...,tf_race_ethnicity_r,bf_race_ethnicity,bf_tf_adj_race_ethnicity,age_l,age_r,street_number_l,street_number_r,street_name_l,street_name_r,match_key
0,-19.888831,0.000001,__splink__input_table_0,__splink__input_table_1,8603,10238,Heidi,Danielle,0,0.000884,...,0.069378,1.714195,4.082625,27,34,1719,8203,sandleheath,west farwell avenue,8
1,-19.105670,0.000002,__splink__input_table_0,__splink__input_table_1,7048,10240,None,Sophia,-1,NaN,...,0.452425,1.714195,0.626056,6,18,308,8203,trowbridge,west farwell avenue,8


We can see how our model performs at different probability thresholds, with a couple of options depending on the space we wish to view things


In [18]:
linker.evaluation.accuracy_analysis_from_labels_column(
    "simulant_id", output_type="accuracy"
)

Blocking time: 0.10 seconds
Predict time: 1.76 seconds


alt.LayerChart(...)

and we can easily see how many individuals we identify and link by looking at clusters generated at some threshold match probability of interest - in this example 99%


In [19]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    predictions, threshold_match_probability=0.99
)
df_clusters = clusters.as_pandas_dataframe().sort_values("cluster_id")
df_clusters.groupby("cluster_id").size().value_counts()

Completed iteration 1, num representatives needing updating: 0


1    10159
2       68
3        2
4        1
Name: count, dtype: int64

In [20]:
linker.visualisations.cluster_studio_dashboard(
    predictions,
    clusters,
    "cluster_studio.html",
    sampling_method="by_cluster_size",
    overwrite=True
)

# You can view the scv.html file in your browser, or inline in a notbook as follows
from IPython.display import IFrame

#IFrame(src="./cluster_studio.html", width="100%", height=1000)

In [21]:
linker.visualisations.comparison_viewer_dashboard(predictions, "scv.html", overwrite=True)
#IFrame(src="./scv.html", width="100%", height=1000)

In this case, we happen to know what the true links are, so we can manually inspect the ones that are doing worst to see what our model is not capturing - i.e. where we have false negatives.

Similarly, we can look at the non-links which are performing the best, to see whether we have an issue with false positives.

Ordinarily we would not have this luxury, and so would need to dig a bit deeper for clues as to how to improve our model, such as manually inspecting records across threshold probabilities,


In [22]:
# add simulant_id_l and simulant_id_r columns by looking up id_l and id_r in the simulant id lookup table
df_predictions = pd.merge(df_predictions, simulant_lookup, left_on="id_l", right_on="id", how="left")
df_predictions = df_predictions.rename(columns={'simulant_id': 'simulant_id_l'})
df_predictions = pd.merge(df_predictions, simulant_lookup, left_on="id_r", right_on="id", how="left")
df_predictions = df_predictions.rename(columns={'simulant_id': 'simulant_id_r'})

In [23]:
# sort links by lowest match_probability to see if we missed any
links = df_predictions[df_predictions["simulant_id_l"]==df_predictions["simulant_id_r"]].sort_values("match_weight")
# sort nonlinks by highest match_probability to see if we matched any
nonlinks = df_predictions[df_predictions["simulant_id_l"]!=df_predictions["simulant_id_r"]].sort_values("match_weight", ascending=False)
display(links)
display(nonlinks)

,match_weight,match_probability,source_dataset_l,source_dataset_r,id_l,id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,...,bf_tf_adj_race_ethnicity,age_l,age_r,street_number_l,street_number_r,street_name_l,street_name_r,match_key,simulant_id_l,simulant_id_r
19908,7.292779,0.993663,__splink__input_table_0,__splink__input_table_1,10228,10282,Benjamin,Benjamin,3,0.005599,...,1.000000,19,19,2002,8203,203rd pl se,west farwell avenue,0,0_19556,0_19556
22350,17.203987,0.999993,__splink__input_table_0,__splink__input_table_1,1990,10298,Male,Lindsey,0,0.000295,...,0.626056,23,23,8203,8203,west farwell avenue,west farwell avenue,1,0_19697,0_19697
2459,17.550639,0.999995,__splink__input_table_0,__splink__input_table_1,1894,10246,Robert,Robert,3,0.007760,...,1.000000,19,19,8203,8203,west farwell avenue,west farwell avenue,0,0_7585,0_7585
18656,17.951793,0.999996,__splink__input_table_0,__splink__input_table_1,1980,10294,Marvln,Woman,0,0.000098,...,0.626056,46,46,8203,8203,west farwell avenue,est farel avenue,1,0_19676,0_19676
23881,18.411753,0.999997,__splink__input_table_0,__splink__input_table_1,1890,10244,Betty,Betty,3,0.003929,...,0.626056,88,88,8203,8203,west farwell avenue,west farwell avenue,0,0_7003,0_7003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18029,40.381846,1.000000,__splink__input_table_0,__splink__input_table_1,1926,10261,Erica,Erica,3,0.001473,...,1.748638,27,27,8203,8203,west farwell avenue,west farwell avenue,0,0_19433,0_19433
18525,41.042397,1.000000,__splink__input_table_0,__splink__input_table_1,1957,10284,Beth,Beth,3,0.000589,...,3.240469,51,51,8203,8203,west farwell avenue,west farwell avenue,0,0_19576,0_19576
17658,41.067758,1.000000,__splink__input_table_0,__splink__input_table_1,1953,10279,Lisamarie,Lisamarie,3,0.000196,...,1.000000,22,22,8203,8203,west farwell avenue,west farwell avenue,0,0_19533,0_19533
17620,42.203005,1.000000,__splink__input_table_0,__splink__input_table_1,1913,10254,Joel,Joel,3,0.000295,...,2.242435,18,18,8203,8203,west farwell avenue,west farweill avenue,0,0_18353,0_18353


,match_weight,match_probability,source_dataset_l,source_dataset_r,id_l,id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,...,bf_tf_adj_race_ethnicity,age_l,age_r,street_number_l,street_number_r,street_name_l,street_name_r,match_key,simulant_id_l,simulant_id_r
10539,17.951236,9.999961e-01,__splink__input_table_0,__splink__input_table_1,5783,10232,Jamie,Anastasia,0,0.001473,...,1.000000,42,47,820,820,cameron road,cameron road,1,0_12123,0_12125
17897,2.631780,8.610706e-01,__splink__input_table_0,__splink__input_table_1,1945,10268,Ian,Ian,3,0.001866,...,4.082625,23,22,8203,8203,west farwell avenue,west farwell avenue,0,0_19497,0_19462
14283,-1.306355,2.879223e-01,__splink__input_table_0,__splink__input_table_1,8102,10249,Craig,Cynthia,0,0.001375,...,2.242435,22,21,904,8203,county road 755a,west farwell avenue,1,0_10883,0_10884
18441,-1.599607,2.481016e-01,__splink__input_table_0,__splink__input_table_1,6536,10260,Miesha,Amanda,0,0.000098,...,1.000000,45,20,nan,8203,cornell street,west farwell avenue,1,0_15660,0_19429
24194,-1.656731,2.407890e-01,__splink__input_table_0,__splink__input_table_1,1946,10291,Anthony,Anthony,3,0.004813,...,1.000000,22,25,8203,8203,west farwell avenue,west farwell avenue,0,0_19501,0_19663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11923,-36.118645,1.340308e-11,__splink__input_table_0,__splink__input_table_1,1899,10294,Cynthia,Woman,0,0.003733,...,1.000000,21,46,8203,8203,west farwell avenue,est farel avenue,7,0_10884,0_19676
12364,-36.118645,1.340308e-11,__splink__input_table_0,__splink__input_table_1,1888,10294,Emma,Woman,0,0.001473,...,1.000000,23,46,8203,8203,west farwell avenue,est farel avenue,7,0_6099,0_19676
8268,-36.118645,1.340308e-11,__splink__input_table_0,__splink__input_table_1,1870,10294,Kari,Woman,0,0.000491,...,1.000000,36,46,8203,8203,west farwell avenue,est farel avenue,7,0_727,0_19676
8203,-36.118645,1.340308e-11,__splink__input_table_0,__splink__input_table_1,1876,10294,Keara,Woman,0,0.000196,...,1.000000,20,46,8203,8203,west farwell avenue,est farel avenue,7,0_2757,0_19676


The lowest match probability for a true link is well above a .99 threshold, so we do not have any false negatives. The small size of the ACS dataset reduces the chances for noise to make linking difficult. For more realistic challenges, try using a larger dataset like W2, or further increasing the noise levels.

For true nonlinks, there is a match probability above .99, indicating a false positive, as well as another comparison which we may consider a match depending on our match threshold, and others with match probabilities which are not insignificant.

We can also view the true links and nonlinks as waterfall charts to see how we ended up at our final match weights, and the ROC chart showing false positives and negatives using match thresholds.

In [24]:
records_to_view = 5
linker.visualisations.waterfall_chart(
    links.head(records_to_view).to_dict(orient="records")
)

alt.LayerChart(...)

In [25]:
records_to_view = 10
linker.visualisations.waterfall_chart(
    nonlinks.head(records_to_view).to_dict(orient="records")
)

alt.LayerChart(...)

In [26]:
linker.evaluation.accuracy_analysis_from_labels_column('simulant_id', output_type="roc")

Blocking time: 0.10 seconds
Predict time: 0.16 seconds


alt.Chart(...)

We may wish to evaluate the effects of using term frequencies for some columns, such as `address`, by looking at examples of the values `tf_address` for both common and uncommon address values.

In [27]:
common_addr = "8203 west farwell avenue  Anytown WA 00000"
records_to_view = 5
linker.visualisations.waterfall_chart(
    links[links["address_l"]==common_addr].head(records_to_view).to_dict(orient="records")
)



alt.LayerChart(...)

In [28]:
uncommon_addr = "820 cameron road  Anytown WA 00000"
records_to_view = 5
linker.visualisations.waterfall_chart(
    links[links["address_l"]==uncommon_addr].head(records_to_view).to_dict(orient="records")
)

alt.LayerChart(...)